## Installation du package selenium et du chromedriver

In [7]:
# installation du package selenium
!pip install selenium
# installation du chormedriver
!apt install chromium-chromedriver
# copie du chromedriver
!cp C:\Users\Adib\Documents\Stage\Stage 1A\Selenium\chromedriver.exe

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (101.0.4951.64-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
cp: cannot stat 'C:UsersAdibDocumentsStageStage': No such file or directory


## Importation des librairies nécessaires

In [8]:
from selenium import webdriver
import pandas as pd

## Modification des options du chromedriver

In [9]:
# options chromedriver pour l'utiliser dans un notebook
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

## Instanciation du WebDriver de Chrome

In [10]:
# création de l'instance WebDriver de Chrome
driver = webdriver.Chrome('chromedriver', options = options)
# lien du site Morocco World News
website = 'https://www.moroccoworldnews.com/'

## Liste des catégories du site

In [12]:
# catégories du site Morocco World News
categories = ['politics', 'economy', 'opinion', 'lifestyle', 'features', 'society', 'education', 'western-sahara', 'environment', 'covid-19', 'um6p', 'ocp', 'maghreb-news', 'middle-east', 'americas', 'europe']

## Récupération des liens d'articles

In [13]:
# liste des liens vers les articles
articles_links = []
# parcours des catégories
for category in categories:
  category_link = website + category
  # parcours des pages de chaque catégorie
  page_number = 0
  while (page_number < 10):
    webpage_link = category_link + '/' + str(page_number)
    page_number += 1
    # parcours des articles de chaque page
    driver.get(webpage_link)
    href_links = driver.find_elements_by_xpath('//h3/a[@href]')
    for href_link in href_links:
      articles_links.append(href_link.get_attribute('href'))

## Vérification de la liste de liens

In [14]:
# affichage de 5 liens pris au hasard
articles_links[95:100]

['https://www.moroccoworldnews.com/2022/04/348482/arab-league-commends-moroccos-position-on-israeli-attacks-on-al-aqsa',
 'https://www.moroccoworldnews.com/2022/06/349620/foa-details-food-security-challenges-for-morocco',
 'https://www.moroccoworldnews.com/2022/06/349619/moroccos-chari-fully-acquires-ivorian-retailer-diago',
 'https://www.moroccoworldnews.com/2022/06/349613/uae-seeks-trilateral-trade-agreement-with-morocco-israel',
 'https://www.moroccoworldnews.com/2022/06/349611/three-moroccan-cities-feature-among-africas-15-most-expensive-cities']

In [15]:
# affichage du nombre total de liens
len(articles_links)

1357

## Récupération du contenu des articles

In [16]:
# data frame des données collectés
df_articles = pd.DataFrame()
# parcours des liens collectés
for article_link in articles_links:
  # accès à la page de l'article
  driver.get(article_link)
  # récupération du titre de l'article
  title_scrap = driver.find_elements_by_class_name('detail_title')
  if (len(title_scrap) != 0):
      title = title_scrap[0].text
  # récupération de l'en-tête de l'article
  lead_scrap = driver.find_elements_by_class_name('detail_lead')
  if (len(lead_scrap) != 0):
    lead = lead_scrap[0].text
  # récupération de l'auteur de l'article
  author_scrap = driver.find_elements_by_xpath('//*[@id="app"]/section[1]/div/div/div/div/div/div[1]/div[1]/div[1]/ul/li[1]/a/span')
  if (len(author_scrap) != 0):
    author = author_scrap[0].text
  # récupération de la date de publication de l'article
  date_scrap = driver.find_elements_by_xpath('//*[@id="app"]/section[1]/div/div/div/div/div/div[1]/div[1]/div[1]/ul/li[2]/span')
  if (len(date_scrap) != 0):
    date = date_scrap[0].text
  # récupération du contenu de l'article
  content_scrap = driver.find_elements_by_class_name('article__content')
  if (len(content_scrap) != 0):
    content = content_scrap[1].text
  # création de la liste de nos données
  article = {'title' : title, 'lead' : lead, 'author' : author, 'date' : date, 'content' : content}
  # transformation en data frame
  df_article = pd.DataFrame(article, index = [0])
  # transformation en data frame
  df_articles = df_articles.append(df_article, ignore_index=True)

## Vérification des données collectés

In [17]:
# affichage des premières valeurs
df_articles.head()

,title,lead,author,date,content
0,"Spanish FM: Spain Works on ‘Constructive, Firm...",Algeria decided to suspend its friendship trea...,Safaa Kasraoui,"June 09, 2022 10:53 a.m.",Rabat - Spain regrets Algeria’s decision to su...
1,"Afro-Atlantic Treaty: Morocco Says Only Unity,...",Morocco’s Foreign Affairs Minister insists tha...,Aya Benazizi,"June 08, 2022 5:08 p.m.",Casablanca - With the Moroccan capital of Raba...
2,Cape Verde to Open Consulate General in Morocc...,Togo also announced the forthcoming opening of...,Safaa Kasraoui,"June 08, 2022 4:40 p.m.",Rabat - Cape Verde’s Foreign Affairs Minister ...
3,"Mexican Delegation To Visit Morocco, Keen To C...","Migration management, the fight against climat...",Souad Anouar,"June 08, 2022 4:20 p.m.",Rabat - A Mexican parliamentary delegation is ...
4,Spanish PM Renews Support for Morocco’s Autono...,A number of marginal opposition political part...,Safaa Kasraoui,"June 08, 2022 2:02 p.m.",Rabat - Spain’s Prime Minister Pedro Sanchez h...


In [18]:
# affichage de la taille de ntore data frame
len(df_articles)

1357

## Conversion en fichier csv

In [19]:
# transformation du data frame en fichier csv
df_articles.to_csv('morocco_world_news_articles.csv')